In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import numpy as np
import pandas as pd

In [2]:
### Load the trained model, scaler pickle,onehot
model=load_model('churn_model.h5')

## load the encoder and scaler
with open('onehotencoder.pkl','rb') as f:
    label_encoder_geo=pickle.load(f)

with open('labelencoder.pkl','rb') as f:
    label_encoder_gender=pickle.load(f)

with open('standardscaler.pkl','rb') as f:
    scaler=pickle.load(f)

In [40]:
input_data={
'CreditScore': 600,
'Geography': 'France',
'Gender':'Male',
'Age': 40,
'Tenure': 3,
'Balance': 60000,
'NumOfProducts': 2,
'HasCrCard': 1,
'IsActiveMember': 1,
'EstimatedSalary': 50000
}

In [41]:
# One hot encoding for Geography
geo_encoded=label_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df=pd.DataFrame(geo_encoded,columns=label_encoder_geo.categories_[0])

In [42]:
geo_encoded_df

,France,Germany,Spain
0,1.0,0.0,0.0


In [43]:
input_df=pd.DataFrame(input_data,index=[0])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [44]:
##Encode categorical variables
input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [45]:
##COncatination one hot encoded 
input_df = pd.concat([input_df.drop('Geography',axis=1),geo_encoded_df],axis=1)

In [46]:
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,France,Germany,Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [47]:
#Scaling the input data
input_scaled=scaler.transform(input_df)
input_scaled

array([[-0.52111599,  0.91601335,  0.10961719, -0.68538967, -0.2569057 ,
         0.8095029 ,  0.64259497,  0.9687384 , -0.87203322,  0.98560362,
        -0.5698444 , -0.57369368]])

In [48]:
#Predict the churn
prediction = model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 197ms/step


array([[0.03198235]], dtype=float32)

In [49]:
prediction_probability = prediction[0][0]
prediction_probability

np.float32(0.03198235)

In [50]:
if prediction_probability > 0.5:
    print("The customer is likely to churn")
else:
    print("The customer is not likely to churn")

The customer is not likely to churn
